### Importing the Libraries

In [29]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [30]:
tf.__version__ 

'2.12.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [31]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [32]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [33]:
!wget https://github.com/AvijitChowdhury/Sign-Language-To-Text-Conversion1/archive/refs/heads/main.zip



--2023-04-01 17:40:17--  https://github.com/AvijitChowdhury/Sign-Language-To-Text-Conversion1/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/AvijitChowdhury/Sign-Language-To-Text-Conversion1/zip/refs/heads/main [following]
--2023-04-01 17:40:18--  https://codeload.github.com/AvijitChowdhury/Sign-Language-To-Text-Conversion1/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip.1’

main.zip.1              [              <=>   ] 421.79M  26.3MB/s    in 16s     

2023-04-01 17:40:34 (25.7 MB/s) - ‘main.zip.1’ saved [442279055]



In [34]:

!unzip main.zip

Archive:  main.zip
5d90dd4f608245cabf41f540a12fd9b1a2f3a327
replace Sign-Language-To-Text-Conversion1-main/dataSet/testingData/0/108.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### Creating the Training set

In [35]:
!mkdir trainingData

mkdir: cannot create directory ‘trainingData’: File exists


In [36]:
!mkdir testingData

mkdir: cannot create directory ‘testingData’: File exists


In [37]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/T' 'trainingData'

In [38]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/K' 'trainingData'

In [39]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/D' 'trainingData'

In [40]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/trainingData/I' 'trainingData'

In [41]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/testingData/T' 'testingData'

In [42]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/testingData/K' 'testingData'

In [43]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/testingData/D' 'testingData'

In [44]:
!cp -r '/content/Sign-Language-To-Text-Conversion1-main/dataSet/testingData/I' 'testingData'

In [45]:
training_set = train_datagen.flow_from_directory('trainingData',                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 1896 images belonging to 4 classes.


In [46]:
test_set = test_datagen.flow_from_directory('testingData',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 631 images belonging to 4 classes.


# Part 2: Building CNN


# Initializing CNN

In [47]:
classifier = tf.keras.models.Sequential()

#Step 1: Convolution

In [48]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#Step 2:  Pooling

In [49]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

# Adding a second convolution layer

In [50]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

# Step 3: Flattering

In [51]:
classifier.add(tf.keras.layers.Flatten())

# Step 4: Full Connection

In [59]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=32, activation='relu')) # softmax for more than 2
classifier.add(tf.keras.layers.Dense(units=16, activation='relu')) # softmax for more than 2
classifier.add(tf.keras.layers.Dense(units=4, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [60]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [61]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 32768)             0         
                                                                 
 dense_5 (Dense)             (None, 128)              

In [62]:
classifier

In [63]:
classifier.fit(training_set,
                  epochs = 30,
                  validation_data = test_set)

Epoch 1/20
190/190 [==============================] - 59s 297ms/step - loss: 0.6287 - accuracy: 0.6719 - val_loss: 0.3533 - val_accuracy: 0.7544
Epoch 2/20
190/190 [==============================] - 52s 272ms/step - loss: 0.4726 - accuracy: 0.7352 - val_loss: 0.3720 - val_accuracy: 0.7401
Epoch 3/20
190/190 [==============================] - 54s 286ms/step - loss: 0.4278 - accuracy: 0.7474 - val_loss: 0.3567 - val_accuracy: 0.7544
Epoch 4/20
190/190 [==============================] - 52s 272ms/step - loss: 0.3936 - accuracy: 0.7220 - val_loss: 0.3538 - val_accuracy: 0.7544
Epoch 5/20
190/190 [==============================] - 53s 279ms/step - loss: 0.3715 - accuracy: 0.7569 - val_loss: 0.3530 - val_accuracy: 0.7544
Epoch 6/20
190/190 [==============================] - 51s 267ms/step - loss: 0.3791 - accuracy: 0.7468 - val_loss: 0.3681 - val_accuracy: 0.7496
Epoch 7/20
190/190 [==============================] - 52s 272ms/step - loss: 0.4192 - accuracy: 0.7521 - val_loss: 0.3557 - val_ac

#### Saving the Model

In [64]:
model_json = classifier.to_json()
with open("model_bw-tkdi.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_bw-tkdi.h5')
print('Weights saved')

Model Saved
Weights saved


In [58]:
classifier.fit(training_set,
                  epochs = 30,
                  validation_data = test_set)

Epoch 1/30
190/190 [==============================] - 51s 269ms/step - loss: 0.4187 - accuracy: 0.7474 - val_loss: 0.3883 - val_accuracy: 0.7544
Epoch 2/30
190/190 [==============================] - 54s 284ms/step - loss: 0.3894 - accuracy: 0.7537 - val_loss: 0.3847 - val_accuracy: 0.7544
Epoch 3/30
190/190 [==============================] - 53s 279ms/step - loss: 0.3946 - accuracy: 0.7511 - val_loss: 0.3816 - val_accuracy: 0.7544
Epoch 4/30
190/190 [==============================] - 51s 271ms/step - loss: 0.3925 - accuracy: 0.7516 - val_loss: 0.3789 - val_accuracy: 0.7544
Epoch 5/30
190/190 [==============================] - 52s 276ms/step - loss: 0.3849 - accuracy: 0.7526 - val_loss: 0.3765 - val_accuracy: 0.7544
Epoch 6/30
190/190 [==============================] - 51s 270ms/step - loss: 0.3905 - accuracy: 0.7511 - val_loss: 0.3744 - val_accuracy: 0.7544
Epoch 7/30
190/190 [==============================] - 54s 286ms/step - loss: 0.4123 - accuracy: 0.7421 - val_loss: 0.3730 - val_ac